In [1]:
import numpy as np

In [68]:
# проверка положительных оценок
def continue_solve(mark_in):
    score = np.copy(mark_in)
    score = score[1:]
    for i in score:
        if i > 0:
            return True,
    return False

# вычисление оценки
def get_mark(matrix, function, basis):
    c_basis = []
    for i in basis:
        c_basis.append(function[i - 1])
    score = np.dot(c_basis, matrix) - (np.append([0], function))
    return score

# получение базиса
def get_basis(matrix):
    basis = []
    for i in range(len(matrix)):
        basis.append(matrix.shape[1] - len(matrix) + i)
    return basis

# добавление переменных к матрице и функции
def add_slack_variables(matrix, function):
    matrix = np.concatenate((matrix, np.eye(matrix.shape[0])), axis=1)
    function = np.append(function, matrix.shape[0] * [0])
    return matrix, function

# пересчет мартрицы
def recount(matrix_in, index_input, index_output):
    matrix = matrix_in.copy()
    k = matrix[index_output][index_input]
    matrix[index_output] /= k

    for i in range(len(matrix)):
        if i != index_output:
            matrix[i] -= matrix[i][index_input] * matrix[index_output]
    return matrix


def get_index_output(index_input, matrix_in):
    matrix = np.copy(matrix_in)
    p_0 = matrix[:, 0]
    p_i = matrix[:, index_input]

    # exclude division by zero
    p_i[p_i == 0] = -1

    teta = p_0 / p_i
    teta = np.where(teta > 0, teta, np.inf)
    index_output = teta.argmin()

    if teta[index_output] == np.inf:
        raise Exception("Not solution")
    else:
        return index_output


def solve(matrix, function, basis):
    score = get_mark(matrix, function, basis)
    flag = continue_solve(score)

    # main loop
    while flag:

        index_input = np.argmax(score)
        index_output = get_index_output(index_input, matrix)

        matrix = recount(matrix, index_input, index_output)

        basis[index_output] = index_input

        score = get_mark(matrix, function, basis)
        flag = continue_solve(score)

    return matrix, function, basis


def get_canonical_form(a, b, c):
    matrix = np.copy(a)
    vector = np.copy(b)
    function = np.copy(c)

    matrix = np.concatenate((vector.T, matrix), axis=1)
    matrix, function = add_slack_variables(matrix, function)
    basis = get_basis(matrix)

    return matrix, function, basis


def simplex_method(matrix, function, basis):
    matrix, function, basis = solve(matrix, function, basis)
    mark = get_mark(matrix, function, basis)

    p_0 = matrix[:, 0]

    x = np.zeros(len(C))

    for i in range(len(basis)):
        if (basis[i] - 1) < len(C):
            x[basis[i] - 1] = p_0[i]

    print(f"x = {x}")
    print(f"result = {mark[0] * mm_scaler}")

In [67]:
def read_from_file(input_file):
    file =  open(input_file)
    lines = file.read().splitlines()
    ineqs = {'<=','>='}

    objective = lines[0].split()

    if objective[-1] == 'min':
        objective_scaler = 1
    elif objective[-1] == 'max':
        objective_scaler = -1
    else:
        raise AssertionError('ошибка ввода')

    c = list(map(float, objective[:-2]))
    A = []
    b = []
    for line in lines[1:]:
        row = []
        ineq_sign = 0
        for val in line.split():
            if val not in ineqs:
                row.append(float(val))
            elif val in ineqs and ineq_sign == 0:
                if val == '<=':
                    ineq_sign = 1
                else:
                    ineq_sign = -1
        if ineq_sign != 0:
            row = np.array(row) * ineq_sign
        else:
            raise AssertionError('ошибка ввода')
        A.append(row[:-1])
        b.append(row[-1])

    A = np.array(A[:-A[1].shape[0]])
    b = np.array([b[:-A[1].shape[0]]])
    c = np.array(c) * objective_scaler

    return A, b, c , objective_scaler

In [66]:
input_file = 'test.txt'

$36*x_1 +10*x_2+ 60*x_3 → max$

$12*x_1 + 8*x_2 + 4*x_3 \leq 136$

$4*x_1 + 6*x_2 + 12*x_3 \leq 120$

$x_1,x_2,x_3 \geq 0$

In [70]:
A, B, C, mm_scaler = read_from_file(input_file)

mat, fun, bas = get_canonical_form(A, B, C)
simplex_method(mat, fun, bas)

x = [9. 0. 7.]
result = 744.0
